In [1]:
import cv2
import numpy as np
import tensorflow as tf

In [2]:
# Assuming you have a trained ResNet model for feature extraction
resnet_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, pooling='avg')

# Assuming you have a trained LSTM model for prediction
lstm_model = tf.keras.models.load_model('best_lstm_model.h5')


In [3]:
def preprocess_frame(frame):
    # Resize to 224x224
    frame_resized = cv2.resize(frame, (224, 224))
    # Convert frame to RGB as ResNet expects RGB format
    frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
    # Preprocess the frame for ResNet
    frame_preprocessed = tf.keras.applications.resnet50.preprocess_input(frame_rgb)
    return frame_preprocessed

def extract_features(video_path, fps=5, duration=10):
  
    cap = cv2.VideoCapture(video_path)
    
    # Calculate the number of frames to extract based on fps and duration
    frames_to_extract = fps * duration  # For a 10-second video at 5 FPS, 50 frames
    
    frame_count = 0
    saved_frame_count = 0
    feature_list = []
    
    while True:
        ret, frame = cap.read()
        if not ret or saved_frame_count >= frames_to_extract:
            break  # End of video or extracted enough frames
        
        # Extract frames at the specified fps rate
        if frame_count % int(cap.get(cv2.CAP_PROP_FPS) // fps) == 0:
            # Preprocess the frame for ResNet
            preprocessed_frame = preprocess_frame(frame)
            preprocessed_frame = np.expand_dims(preprocessed_frame, axis=0)  # Add batch dimension
            
            # Extract features using ResNet
            features = resnet_model.predict(preprocessed_frame)
            feature_list.append(features[0])  # Add the extracted features for the frame
            
            saved_frame_count += 1
        
        frame_count += 1
    
    cap.release()  # Release video capture object
    
    # Convert list of features to numpy array and return
    if saved_frame_count > 0:
        feature_array = np.array(feature_list)
        return feature_array
    else:
        print("No frames extracted from the video.")
        return None

def predict_engagement(features, lstm_model):
    
    # Reshape features for LSTM model (add batch dimension)
    features = np.expand_dims(features, axis=0)
    
    # Get the prediction from the LSTM model
    prediction = lstm_model.predict(features)
    
    # Convert the prediction to engagement level (max probability class)
    predicted_engagement = np.argmax(prediction, axis=1)  # Predicted engagement level
    return predicted_engagement[0]

def video_to_engagement(video_path, fps=5, duration=10):
    # Step 1: Extract features from the video
    features = extract_features(video_path, fps, duration)
    # Step 2: If features are extracted, pass them to the LSTM model for prediction
    if features is not None:
        predicted_engagement = predict_engagement(features, lstm_model)
        print(f"Predicted Engagement Level: {predicted_engagement}")
        return predicted_engagement
    else:
        print("No features extracted. Cannot predict engagement.")
        return None

In [4]:

# Example usage
video_path = "20250401_154433.mp4"
predicted_engagement = video_to_engagement(video_path, fps=5, duration=10)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━